## Text to SQL generation using Code-Bison

#### Imports 

In [1]:
from vertexai.language_models import CodeGenerationModel
from vertexai.language_models import TextGenerationModel
from google.cloud import bigquery
import logging 
import os 

##### Setup logging

In [2]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

#### Setup essentials

In [3]:
SERVICE_ACCOUNT_CREDENTIALS = './../credentials/vai-key.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = SERVICE_ACCOUNT_CREDENTIALS

In [4]:
PROJECT_ID = 'arun-genai-bb'
CODE_GEN_MODEL_NAME = 'code-bison@latest'
TEXT_GEN_MODEL_NAME = 'text-bison@latest'
TEMPERATURE = 0 # default value = 0
MAX_OUTPUT_TOKENS = 2048  # length of the output response | overridding the default value which is 128
# TOP_P = 0.95  # default value
# TOP_K = 40  # default value
LOCATION = 'us-central1'

In [5]:
DATASET = 'flight_reservations'
TABLES = ['customers', 'flights', 'reservations', 'transactions', 'loyality_points']

In [6]:
code_gen_model = CodeGenerationModel.from_pretrained(CODE_GEN_MODEL_NAME)
text_gen_model = TextGenerationModel.from_pretrained(TEXT_GEN_MODEL_NAME)
bq_client = bigquery.Client()

In [7]:
query = f"""
    SELECT *
    FROM `{PROJECT_ID}.{DATASET}.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS`
    WHERE table_name in ({','.join([f'"{table}"' for table in TABLES])})
"""
logger.info(query)


    SELECT *
    FROM `arun-genai-bb.flight_reservations.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS`
    WHERE table_name in ("customers","flights","reservations","transactions","loyality_points")



In [8]:
schema_columns = bq_client.query(query=query).to_dataframe()
schema_columns

,table_catalog,table_schema,table_name,column_name,field_path,data_type,description,collation_name,rounding_mode
0,arun-genai-bb,flight_reservations,loyality_points,customer_id,customer_id,INT64,None,NULL,None
1,arun-genai-bb,flight_reservations,loyality_points,points,points,INT64,None,NULL,None
2,arun-genai-bb,flight_reservations,loyality_points,last_updated_datetime,last_updated_datetime,DATETIME,None,NULL,None
3,arun-genai-bb,flight_reservations,transactions,transaction_id,transaction_id,INT64,None,NULL,None
4,arun-genai-bb,flight_reservations,transactions,reservation_id,reservation_id,INT64,None,NULL,None
5,arun-genai-bb,flight_reservations,transactions,amount,amount,FLOAT64,None,NULL,None
6,arun-genai-bb,flight_reservations,transactions,transaction_datetime,transaction_datetime,DATETIME,None,NULL,None
7,arun-genai-bb,flight_reservations,reservations,reservation_id,reservation_id,INT64,None,NULL,None
8,arun-genai-bb,flight_reservations,reservations,customer_id,customer_id,INT64,None,NULL,None
9,arun-genai-bb,flight_reservations,reservations,flight_id,flight_id,INT64,None,NULL,None


In [9]:
schema_columns = schema_columns.to_markdown(index=False)
logger.info(schema_columns)

| table_catalog   | table_schema        | table_name      | column_name           | field_path            | data_type   | description   | collation_name   | rounding_mode   |
|:----------------|:--------------------|:----------------|:----------------------|:----------------------|:------------|:--------------|:-----------------|:----------------|
| arun-genai-bb   | flight_reservations | loyality_points | customer_id           | customer_id           | INT64       |               | NULL             |                 |
| arun-genai-bb   | flight_reservations | loyality_points | points                | points                | INT64       |               | NULL             |                 |
| arun-genai-bb   | flight_reservations | loyality_points | last_updated_datetime | last_updated_datetime | DATETIME    |               | NULL             |                 |
| arun-genai-bb   | flight_reservations | transactions    | transaction_id        | transaction_id        | INT64       |    

Utility function to convert text into SQL and automatically correct the SQL query if any execution errors occur.

In [10]:
def generate_and_execute_sql(prompt, max_tries=5):
    """
    Generate an SQL query using the code_gen_model and execute it using bq_client.
    
    Args:
    - prompt (str): Prompt to provide to the model for generating SQL.
    - max_tries (int): Maximum number of attempts to generate and execute SQL.
    
    Returns:
    - dict: A dictionary containing the successful dataframe or error messages and prompt evolution.
    """
    
    tries = 0
    error_messages = []
    prompts = [prompt]
    
    while tries < max_tries:
        logger.info(f'ATTEMPT: {tries+1}')
        try:
            # Predict SQL using the model
            response = code_gen_model.predict(prompt, temperature=TEMPERATURE, max_output_tokens=MAX_OUTPUT_TOKENS)
            generated_sql_query = response.text
            generated_sql_query = '\n'.join(generated_sql_query.split('\n')[1:-1])
            logger.info('-' * 50)
            logger.info(generated_sql_query)
            logger.info('-' * 50)
            # Execute SQL using BigQuery client
            df = bq_client.query(generated_sql_query).to_dataframe()
            logger.info('SUCCEEDED')
            # If successful, return the dataframe
            return {
                "dataframe": df,
                "prompts": prompts,
                "errors": error_messages
            }
        except Exception as e:
            logger.error('FAILED')
            # Catch the error, store the message, and try again
            msg = str(e)
            error_messages.append(msg)
            # Evolve the prompt by appending the error message and asking the model to correct it
            prompt = f"""{prompt}
Encountered an error: {msg}. 
To address this, please generate an alternative SQL query response that avoids this specific error. 
Follow the instructions mentioned above to remediate the error. 

Modify the below SQL query to resolve the issue:
{generated_sql_query}

Provide a detailed explanation of the modifications made and the rationale behind those changes. 
Ensure the revised SQL query aligns precisely with the requirements outlined in the initial question."""
            prompts.append(prompt)
            tries += 1
        logger.info('=' * 100)
    # If all tries exhausted, return the errors
    return {
        "error": "All attempts exhausted.",
        "prompts": prompts,
        "errors": error_messages
    }

### Test text to SQL scenarios

Construct the SEED prompt

In [11]:
seed_prompt = """
Please craft a SQL query for BigQuery that addresses the following QUESTION provided below. 
Ensure you reference the appropriate BigQuery tables and column names provided in the SCHEMA below. 
When joining tables, employ type coercion to guarantee data type consistency for the join columns. 
Additionally, the output column names should specify units where applicable.\n
QUESTION:
{}\n
SCHEMA:
{}\n
IMPORTANT: 
Use ONLY DATETIME and DO NOT use TIMESTAMP.
Ensure your SQL query accurately defines both the start and end of the DATETIME range.
"""
logger.info(seed_prompt)


Please craft a SQL query for BigQuery that addresses the following QUESTION provided below. 
Ensure you reference the appropriate BigQuery tables and column names provided in the SCHEMA below. 
When joining tables, employ type coercion to guarantee data type consistency for the join columns. 
Additionally, the output column names should specify units where applicable.

QUESTION:
{}

SCHEMA:
{}

IMPORTANT: 
Use ONLY DATETIME and DO NOT use TIMESTAMP.
Please ensure that you search for items exclusively within the specified date and time range when asked to.



#### Scenario 1: Retrieve Active Reservations for a Specific Date Range

For this scenario, you want to find all active reservations within a specific date range.

In [12]:
question = "Provide a list of all reservations from October 10th to October 15th, 2023"

In [13]:
prompt = seed_prompt.format(question, schema_columns)
logger.info(prompt)


Please craft a SQL query for BigQuery that addresses the following QUESTION provided below. 
Ensure you reference the appropriate BigQuery tables and column names provided in the SCHEMA below. 
When joining tables, employ type coercion to guarantee data type consistency for the join columns. 
Additionally, the output column names should specify units where applicable.

QUESTION:
Provide a list of all reservations from October 10th to October 15th, 2023

SCHEMA:
| table_catalog   | table_schema        | table_name      | column_name           | field_path            | data_type   | description   | collation_name   | rounding_mode   |
|:----------------|:--------------------|:----------------|:----------------------|:----------------------|:------------|:--------------|:-----------------|:----------------|
| arun-genai-bb   | flight_reservations | loyality_points | customer_id           | customer_id           | INT64       |               | NULL             |                 |
| arun-g

In [14]:
%%time

response = generate_and_execute_sql(prompt=prompt)

ATTEMPT: 1
--------------------------------------------------
SELECT
  r.reservation_id,
  r.customer_id,
  r.flight_id,
  r.reservation_datetime,
  r.status,
  f.origin,
  f.destination,
  f.departure_datetime,
  f.arrival_datetime,
  f.carrier,
  f.price
FROM flight_reservations.reservations r
JOIN flight_reservations.flights f
ON r.flight_id = f.flight_id
WHERE r.reservation_datetime BETWEEN '2023-10-10' AND '2023-10-15'
ORDER BY r.reservation_datetime;
--------------------------------------------------
SUCCEEDED


CPU times: user 97.2 ms, sys: 11.8 ms, total: 109 ms
Wall time: 5.41 s


In [15]:
sql_output = response['dataframe']
sql_output

,reservation_id,customer_id,flight_id,reservation_datetime,status,origin,destination,departure_datetime,arrival_datetime,carrier,price
0,6,6,6,2023-10-10 10:00:00,Confirmed,SEA,JFK,2023-11-25 06:00:00,2023-11-25 14:30:00,United,550.0
1,7,7,7,2023-10-12 11:30:00,Confirmed,JFK,MIA,2023-11-27 20:00:00,2023-11-27 23:30:00,American,380.0


#### Scenario 2: Identify customers who made reservations in the past N days.

In [16]:
question = "Identify all customers who have made flight reservations within the last 7 days."

In [17]:
prompt = seed_prompt.format(question, schema_columns)

In [18]:
%%time

response = generate_and_execute_sql(prompt=prompt)

ATTEMPT: 1
--------------------------------------------------
SELECT
  c.customer_id,
  c.first_name,
  c.last_name,
  c.email,
  r.reservation_id,
  r.flight_id,
  r.reservation_datetime
FROM arun-genai-bb.flight_reservations.reservations r
JOIN arun-genai-bb.flight_reservations.customers c
ON r.customer_id = c.customer_id
WHERE r.reservation_datetime >= DATE_SUB(CURRENT_DATETIME(), INTERVAL 7 DAY)
--------------------------------------------------
SUCCEEDED


CPU times: user 63.3 ms, sys: 11.6 ms, total: 74.9 ms
Wall time: 5 s


In [19]:
sql_output = response['dataframe']
sql_output

,customer_id,first_name,last_name,email,reservation_id,flight_id,reservation_datetime
0,4,Bob,Smith,bob.smith@example.com,4,4,2023-10-05 08:30:00
1,5,Charlie,Brown,charlie.b@example.com,5,5,2023-10-06 14:20:00
2,6,Diana,Prince,diana.p@example.com,6,6,2023-10-10 10:00:00
3,7,Ethan,Hunt,ethan.h@example.com,7,7,2023-10-12 11:30:00
4,8,Fiona,Shrek,fiona.s@example.com,8,8,2023-10-15 13:20:00
5,9,Gary,Oldman,gary.o@example.com,9,9,2023-10-20 09:00:00
6,10,Hannah,Montana,hannah.m@example.com,10,10,2023-10-22 15:45:00
7,11,Ian,Somerhalder,ian.s@example.com,11,11,2023-10-25 12:30:00
8,12,Jack,Sparrow,jack.s@example.com,12,12,2023-10-28 17:10:00
9,13,Kate,Winslet,kate.w@example.com,13,13,2023-10-30 14:50:00
